In [8]:
# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

from params_and_keys import *
# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']

2.8.0


In [15]:
def model_predict(img_path, detector):

    def display_image(image):
        fig = plt.figure(figsize=(20, 15))
        plt.grid(False)
        plt.imshow(image)

    
    def draw_bounding_box_on_image(image,
                                   ymin,
                                   xmin,
                                   ymax,
                                   xmax,
                                   color,
                                   font,
                                   thickness=4,
                                   display_str_list=()):
      """Adds a bounding box to an image."""
      draw = ImageDraw.Draw(image)
      im_width, im_height = image.size
      (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                    ymin * im_height, ymax * im_height)
      draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
                 (left, top)],
                width=thickness,
                fill=color)

      # If the total height of the display strings added to the top of the bounding
      # box exceeds the top of the image, stack the strings below the bounding box
      # instead of above.
      display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
      # Each display_str has a top and bottom margin of 0.05x.
      total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

      if top > total_display_str_height:
        text_bottom = top
      else:
        text_bottom = top + total_display_str_height
      # Reverse list and print from bottom to top.
      for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill="black",
                  font=font)
        text_bottom -= text_height - 2 * margin  
        
    def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
      """Overlay labeled boxes on an image with formatted scores and label names."""
      colors = list(ImageColor.colormap.values())

      try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                                  25)
      except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()

      for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
          ymin, xmin, ymax, xmax = tuple(boxes[i])
          display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                         int(100 * scores[i]))
          color = colors[hash(class_names[i]) % len(colors)]
          image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
          draw_bounding_box_on_image(
              image_pil,
              ymin,
              xmin,
              ymax,
              xmax,
              color,
              font,
              display_str_list=[display_str])
          np.copyto(image, np.array(image_pil))
      return image
    
    def load_img(path):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize([img], (640, 640))
        img = tf.cast(img, dtype = tf.float32)
        img = img/255
        return img


    def run_detector(detector, path):
      img = load_img(path)

      #converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
      start_time = time.time()
      result = detector(img)
      end_time = time.time()

      result = {key:value.numpy() for key,value in result.items()}
      return result
      print("Found %d objects." % len(result["detection_scores"]))
      print("Inference time: ", end_time-start_time)

      image_with_boxes = draw_boxes(
          img.numpy(), result["detection_boxes"],
          result["detection_class_entities"], result["detection_scores"])

      display_image(image_with_boxes)
        
    def detect_img(image_path):
      start_time = time.time()
      res = run_detector(detector, image_path)
      end_time = time.time()
      print("Inference time:",end_time-start_time)
      return (res, end_time)
    
    img_res = detect_img(img_path)
    entities = []
    for i in img_res[0].get("detection_class_entities",[]):
        entities.append(i.decode("utf-8"))
        
    def create_counts_dict(thresholds:list = sorted(THRESHOLDS)):
        
        car_dict = {car_name+'_'+str(threshold):0 for car_name in CAR_NAMES for threshold in THRESHOLDS}
        return car_dict
    
    
    def get_counts_for_thresholds(thresholds:list = sorted(THRESHOLDS)):
        car_dict = create_counts_dict()
        for ind, detection_score in enumerate(img_res[0].get("detection_scores",[])):
            for threshold in thresholds:
                if detection_score >= threshold:
                    vehicle_type = entities[ind]
                    if vehicle_type in CAR_NAMES:
                        car_dict[vehicle_type+'_'+str(threshold)] += 1
        return car_dict

    vehicle_counts = get_counts_for_thresholds()
    
    
    return vehicle_counts

In [16]:
model_predict('traff.jpg', detector)

Inference time: 9.430899858474731


{'Car_0.7': 1,
 'Car_0.5': 8,
 'Car_0.3': 13,
 'Car_0.1': 27,
 'Taxi_0.7': 0,
 'Taxi_0.5': 1,
 'Taxi_0.3': 3,
 'Taxi_0.1': 9,
 'Land vehicle_0.7': 0,
 'Land vehicle_0.5': 0,
 'Land vehicle_0.3': 1,
 'Land vehicle_0.1': 18,
 'Van_0.7': 0,
 'Van_0.5': 0,
 'Van_0.3': 0,
 'Van_0.1': 1,
 'Truck_0.7': 0,
 'Truck_0.5': 0,
 'Truck_0.3': 0,
 'Truck_0.1': 0,
 'Vehicle_0.7': 0,
 'Vehicle_0.5': 0,
 'Vehicle_0.3': 0,
 'Vehicle_0.1': 0,
 'Bus_0.7': 0,
 'Bus_0.5': 0,
 'Bus_0.3': 0,
 'Bus_0.1': 0,
 'Motorcycle_0.7': 0,
 'Motorcycle_0.5': 0,
 'Motorcycle_0.3': 0,
 'Motorcycle_0.1': 0,
 'Ambulance_0.7': 0,
 'Ambulance_0.5': 0,
 'Ambulance_0.3': 0,
 'Ambulance_0.1': 0}

In [9]:

def create_counts_dict(thresholds:list = sorted(THRESHOLDS)):

    car_dict = {car_name+'_'+str(threshold):0 for car_name in CAR_NAMES for threshold in THRESHOLDS}
    return car_dict